# Question 1. dlt Version
___

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```shell
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```

What's the version of dlt that you installed?

In [1]:
from dlt import version
version.get_installed_requirement_string()

'dlt==1.13.0'

**Answer:** `1.13.0`

## dlt Resourse
___

For reading the FAQ data, we have this helper function:

```python
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc
```

Annotate it with `@dlt.resource`. We will use it when creating a dlt pipeline.

# Question 2. dlt pipeline
___

Now let's create a pipeline.

We need to define a destination for that. Let's use the `qdrant` one:

```python
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)
```

In this case, we tell dlt (and Qdrant) to create a folder with our data, and the name for it will be `db.qdrant`.

Let's run it:

```python
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)
```

How many rows were inserted into the zoomcamp_data collection?

Look for `"Normalized data for the following tables:"` in the trace output.

In [3]:
import dlt
import requests
from dlt.destinations import qdrant


@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

qdrant_destination = qdrant(qd_path="db.qdrant")
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-11 01:48:37.225111+00:00 and COMPLETED in 8.89 seconds with 4 steps.
Step extract COMPLETED in 1.02 seconds.

Load package 1752198518.465681 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.06 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752198518.465681 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.57 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.56 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/jovyan/work/module_3/db.qdrant location to store data
Load package 1752198518.465681 is LOADED and contains no failed jobs

Step run COMPLETED in 8.88 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.56 seconds
1 load package(s) were loaded to destination qdrant 

**Answer:** `948`

# Question 3. Embeddings
___

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.

**Answer**: `bge-small-en`